<a href="https://colab.research.google.com/github/geraw/ProvengoDemo/blob/main/ProvegoTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
!wget -nc https://downloads.provengo.tech/unix-dist/deb/Provengo-deb.deb
!apt-get install ./Provengo-deb.deb

File ‘Provengo-deb.deb’ already there; not retrieving.

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'provengo' instead of './Provengo-deb.deb'
provengo is already the newest version (0.7.5-SNAPSHOT).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


# System Under Test

In [ ]:
import socket
import random

host = socket.gethostbyname(socket.gethostname())
print(f"{host=}")

host='172.28.0.12'


In [ ]:
from flask import Flask, request, jsonify
import threading
import time
import os
import sys
import json
import importlib
import signal

port = random.randint(1024, 65535)
url = f"http://{host}:{port}"

# Create Flask app
app = Flask(__name__)
#run_with_ngrok(app)  # Allows access from outside

# In-memory data store (replace with a database in a real application)
users = []
loans = []
holds = []
books = {}

# --- User Routes ---
@app.route('/users', methods=['POST'])
def add_user():
    user = request.get_json()

    if 'id' not in user:
        return jsonify({'error': 'user id is required'}), 400
    if  user.get('id') in [u.get('id') for u in users]:
        return jsonify({'error': 'User already exists'}), 400
    else:
      users.append(user)
      return jsonify({'message': 'User Added', 'user': user}), 201

@app.route('/users/<int:user_id>', methods=['DELETE'])
def delete_user(user_id):
    global users
    users = [user for user in users if user.get('id') != user_id]
    return jsonify({'message': 'User deleted'}), 200

@app.route('/users', methods=['GET'])
def search_users():
    query = request.args.get('q', '').lower()
    results = [user for user in users if query in str(user).lower()] if query else users
    return jsonify(results)

# --- Loan Routes ---
@app.route('/loans', methods=['POST'])
def add_loan():
    loan = request.get_json()
    loans.append(loan)
    return jsonify({'message': 'Loan added', 'loan': loan}), 201

@app.route('/loans/<int:loan_id>', methods=['DELETE'])
def delete_loan(loan_id):
    global loans
    loans = [loan for loan in loans if loan.get('id') != loan_id]
    return jsonify({'message': 'Loan deleted'}), 200

@app.route('/loans', methods=['GET'])
def search_loans():
    query = request.args.get('q', '').lower()
    results = [loan for loan in loans if query in str(loan).lower()] if query else loans
    return jsonify(results)

# --- Hold Routes ---
@app.route('/holds', methods=['POST'])
def add_hold():
    hold = request.get_json()
    holds.append(hold)
    return jsonify({'message': 'Hold added', 'hold': hold}), 201

@app.route('/holds/<int:hold_id>', methods=['DELETE'])
def delete_hold(hold_id):
    global holds
    holds = [hold for hold in holds if hold.get('id') != hold_id]
    return jsonify({'message': 'Hold deleted'}), 200

@app.route('/holds', methods=['GET'])
def search_holds():
    query = request.args.get('q', '').lower()
    results = [hold for hold in holds if query in str(hold).lower()] if query else holds
    return jsonify(results)

# --- Book Routes ---
@app.route('/books', methods=['GET', 'POST'])
def manage_books():
    if request.method == 'GET':
        return jsonify(books)
    elif request.method == 'POST':
        book_data = request.get_json()
        book_id = book_data.get('id')
        if book_id:
            books[book_id] = book_data
            return jsonify({'message': 'Book added'}), 201
        return jsonify({'error': 'Invalid book data'}), 400


threading.Thread(target=app.run, kwargs={'host':host,'port':port}).start()


 * Serving Flask app '__main__'


In [ ]:
import requests

try:
    response = requests.post(url+"/users", json = {"id": 225, "name": "Dror"})
    response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
    print("Successfully accessed the server!")
    print("Response:", response.json())
except requests.exceptions.RequestException as e:
    print(f"Error accessing the server: {e}")


 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://172.28.0.12:41274
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:172.28.0.12 - - [11/Mar/2025 13:36:01] "POST /users HTTP/1.1" 201 -


Successfully accessed the server!
Response: {'message': 'User Added', 'user': {'id': 225, 'name': 'Dror'}}


# Test model

In [ ]:
!provengo --batch-mode create BookStoreDemo
!rm /content/BookStoreDemo/spec/js/hello-world.js

        / / 
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \ 

13:36:03.591 INFO [SETUP] We'd love to hear from you! hello@provengo.tech or https://provengo.tech/feedback/
13:36:03.873 ERR  [SETUP] /content/BookStoreDemo Folder already exist. delete the existing folder or create new project with new name.
13:36:03.874 ERR  [Main] Terminating because of previous errors. See logs.
13:36:03.887 ERR  [Main] 40 BadRequest
rm: cannot remove '/content/BookStoreDemo/spec/js/hello-world.js': No such file or directory


In [ ]:
import os

with open("/content/BookStoreDemo/spec/js/_constants.js", "w") as f:
  f.write(f"""
const {host=};
const {port=};
""")

In [ ]:
%%writefile /content/BookStoreDemo/spec/js/model.js

const svc = new RESTSession("http://"+host+":"+port, "provengo basedclient", {
    headers: {"Content-Type":"application/json"}
});


function validation_func(response){
  const respObj = JSON.parse(response.body);
  if ( respObj.message != "User Added" ) {
      pvg.fail("user creation failed: " + respObj.message);
  } else {
      pvg.log.info(`response is: ${JSON.stringify(respObj)}`);
  }
}


bthread("API User", function(){
    svc.get("/users", {
      callback: function(response){
        const respObj = JSON.parse(response.body);
        pvg.log.info(`response is: ${JSON.stringify(respObj)}`);
      }
    })

    svc.post("/users", {
        body: JSON.stringify({
            id: 111,
            name: "John Doe",
        }),
        callback: validation_func
    });

    svc.get("/users")
});

Overwriting /content/BookStoreDemo/spec/js/model.js


In [ ]:
!provengo run BookStoreDemo

        / / 
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \ 

13:36:15.631 INFO [SETUP] We'd love to hear from you! hello@provengo.tech or https://provengo.tech/feedback/
13:36:15.680 INFO [SETUP] Project path: /content/BookStoreDemo
13:36:15.710 INFO [RUN] Preparing to run
13:36:16.023 INFO [RUN>BUILD] model.js: Library 'REST' summoned automatically. Add "//@provengo summon rest" to the top of the file to explicitly summon it. Set the auto-summon configuration key to false to disable this behavior
13:36:16.824 INFO [RUN>random] B-program started
13:36:16.921 INFO [RUN>random] Selected: [GET {lib:"REST", method:"GET", url:"http://172.28.0.12:41274/users", headers:{Content-Type:"application/json"}, parameters:{}, expect

INFO:werkzeug:172.28.0.12 - - [11/Mar/2025 13:36:17] "GET /users HTTP/1.1" 200 -


13:36:17.421 INFO [RUN>random] response is: [{"id":225,"name":"Dror"}]


INFO:werkzeug:172.28.0.12 - - [11/Mar/2025 13:36:17] "POST /users HTTP/1.1" 201 -
INFO:werkzeug:172.28.0.12 - - [11/Mar/2025 13:36:18] "GET /users HTTP/1.1" 200 -


13:36:17.842 INFO [RUN>random] Selected: [POST {lib:"REST", method:"POST", url:"http://172.28.0.12:41274/users", headers:{Content-Type:"application/json"}, parameters:{}, expectedResponseCodes:[], callback:{}, body:"{"id":111,"name":"John Doe"}"}]
13:36:17.924 INFO [RUN>random] response is: {"message":"User Added","user":{"id":111,"name":"John Doe"}}
13:36:18.008 INFO [RUN>random] Selected: [GET {lib:"REST", method:"GET", url:"http://172.28.0.12:41274/users", headers:{Content-Type:"application/json"}, parameters:{}, expectedResponseCodes:[], callback:{}}]
13:36:18.190 INFO [RUN] Test Result: SUCCESS
